<a href="https://colab.research.google.com/github/kuzovkov/neural-university/blob/main/%D0%91%D0%B0%D0%B7%D0%B0_%D0%B7%D0%BD%D0%B0%D0%BD%D0%B8%D0%B9_%7C_%D0%9E%D0%BF%D0%B5%D1%80%D0%B0%D1%82%D0%BE%D1%80_with_%7C_%D0%A3%D0%98%D0%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Оператор **with** и запись в файл

Оператор `with` в Python используется при обработке исключений, чтобы сделать код более понятным и читабельным. Это упрощает управление общими ресурсами, такими как файловые потоки.

Предположим, необходимо создать файл со строкой внутри. Сделать это можно следующим образом:

1. Открытие (создание) файла для записи
2. Запись строки в файл
3. Закрытие файла

Если в режиме записи (`'w'`) файла нет, то `open()` автоматически создает его.

In [ ]:
# Открытие файла для записи
file = open('my_file.txt', 'w')

In [ ]:
# Запись строки в файл
file.write('Hello, world!')

# Закрытие файла
file.close()

Проверка содержания файла командой `!cat`:

In [ ]:
!cat my_file.txt

Hello, world!

> Закрывать файл необходимо. Иначе в коде могут возникнуть ошибки, а многие изменения в файле не вступят в силу.

Между открытием и закрытием файла – например, при записи строки `file.write('Hello, world!')` – может произойти непредвиденная ошибка (да, в данном простом примере это навряд-ли, но ошибки как раз-таки и возникают там, где их не ждешь). Тогда компилятор попросту не дойдет до закрытия файла.

* Что же с этим делать?

Можно воспользоваться конструкцией try-finally:


In [ ]:
# Открытие файла для записи
file = open('my_file.txt', 'w')

# Попытаться записать строку в файл
try:
  file.write('Hello, world!')

# В любом случае закрыть файл
finally:
  file.close()

Проверка содержания файла:

In [ ]:
!cat my_file.txt

Hello, world!

В таком случае при возникновении ошибки в теле `try` закрытие файла все равно произойдет.

> Вопрос: *Зачем нужен `finally`? Разве недостаточно просто `try`?*<br>
Ответ: `try` – это часть *конструкции*. Это значит, что `try` употребляется в связке с другими ключевыми словами  – `except` или `finally`. Без них компилятор выдаст синтаксическую ошибку.

Однако есть более лаконичный вариант с использованием `with`.

In [ ]:
# Открыть файл для записи и закрыть его после выполнения тела with
with open('my_file.txt', 'w') as file:

  # Запись строку в файл
  file.write('Hello, world!')

Проверка содержания файла:

In [ ]:
!cat my_file.txt

Hello, world!

С `with` не приходится прописывать закрытие. Файл записывается в переменную, указанную после ключевого слова `as`. Закрытие файла "заложено" во внутрь `with` и безусловно происходит после выполнения тела `with`. Таким образом можно не думать о проблеме незакрытого файла, а код становится красивей.

> Использование оператора `with` делает код более чистым и безопасным.

Сама инструкция `with` обеспечивает правильное получение и высвобождение ресурсов. Исключение во время вызова `file.write()` в первой реализации может помешать правильному закрытию файла, что может привести к нескольким ошибкам в коде, т.е. многие изменения в файлах не вступят в силу, пока файл не будет должным образом закрыт.

Таким образом, оператор `with` помогает избежать ошибок и утечек, гарантируя, что ресурс правильно высвобождается, когда код, использующий ресурс, полностью выполняется.

## Контекстный менеджер

В `open()` нет ничего особенного, что позволяет использовать его (и только его) с оператором `with`. Этот тип объекта еще называют *контекстным менеджером*. Контекстный он потому, что в теле `with` открытие файла подразумевает его закрытие, а значит, выполняется автоматически, "под капотом".

Можно написать собственный объект, поддерживаемый `with`. Поддержка оператора `with` в ваших объектах гарантирует, что вы никогда не оставите какие-либо ресурсы открытыми.

Чтобы использовать оператор `with` в пользовательском контекстном менеджере, вам нужно только добавить реализации методов `__enter __()` и `__exit __()` в объект. Для дальнейшего пояснения рассмотрим следующий пример.

In [ ]:
# Простой объект записи файла
class OwnFileWriter:

  # Метод инициализации принимает на вход путь к файлу
  def __init__(self, file_name):

    # Сохранение пути к файлу в поле класса
    self.file_name = file_name

  # Метод контекста выполнения
  def __enter__(self):

    # Инициализация нового поля класса файлом в режиме записи
    self.file = open(self.file_name, 'w')

    # Возвращаемое значение присваивается переменной, следующей после as
    return self.file

  # Метод контекста закрытия тела with.
  def __exit__(self, exc_type, exc_value, traceback):

    # Закрытие файла
    self.file.close()

Здесь:
* `exc_type` – тип зафиксированного исключения, либо None.

* `exc_value` – объект зафиксированного исключения, либо None.

* `traceback` – трассировка стека для зафиксированного исключения, либо None.

Без этих 3-х аргументов в объявлении функции возникает ошибка, поэтому их нужно объявлять.

При использовании инструкции `with` результат работы метода `__enter__()`, будет доступен в части `as`.

Теперь собственный контекстный менеджер готов к использованию с `with`.

In [ ]:
with OwnFileWriter('my_file.txt') as file:
    file.write('Hello, world!')

Проверим содержимое файла:

In [ ]:
!cat my_file.txt

Hello, world!